# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray
`

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pprint
from pymongo import UpdateMany

In [5]:
# Create an instance of MongoClient
client = MongoClient(port=27017)


In [6]:
# confirm that our new database was created
print(client.list_database_names())


['admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [7]:
# assign the uk_food database to a variable name
db = client['uk_food']


In [8]:
# review the collections in our new database
print(db.list_collection_names())


['establishments']


In [9]:
# review a document in the establishments collection
document = db.establishments.find_one()
pprint.pprint(document)


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67ebf1085126812def359ef6'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'longi

In [10]:
# assign the collection to a variable
establishments = db['establishments']


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [14]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)
print("Restaurant inserted successfully.")


Restaurant inserted successfully.


In [15]:
# Check that the new restaurant was inserted
inserted_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint.pprint(inserted_restaurant)


{'AddressLine1': '123 Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 'CuisineType': 'Halal',
 'PostCode': 'SE10 8JL',
 'RatingValue': 'Not Yet Rated',
 '_id': ObjectId('67ebf7bfc9882ef50e5bc1f8'),
 'latitude': None,
 'longitude': None}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [17]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"}, 
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})
pprint.pprint(business_type)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [19]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = 1

if business_type_id:
    establishments.update_one(
        {"BusinessName": "Penang Flavours"}, 
        {"$set": {"BusinessTypeID": business_type_id}}
    )
    print("Restaurant updated successfully.")
else:
    print("BusinessTypeID not found.")


Restaurant updated successfully.


In [20]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint.pprint(updated_restaurant)


{'AddressLine1': '123 Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 'CuisineType': 'Halal',
 'PostCode': 'SE10 8JL',
 'RatingValue': 'Not Yet Rated',
 '_id': ObjectId('67ebf7bfc9882ef50e5bc1f8'),
 'latitude': None,
 'longitude': None}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [22]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")


Number of documents with LocalAuthorityName 'Dover': 0


In [23]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Deleted {result.deleted_count} documents with LocalAuthorityName 'Dover'.")


Deleted 0 documents with LocalAuthorityName 'Dover'.


In [24]:
# Check if any remaining documents include Dover
remaining_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Documents with exact 'Dover' match: {remaining_count}")


Documents with exact 'Dover' match: 0


In [25]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
if remaining_document:
    pprint.pprint(remaining_document)
else:
    print("Collection is empty")

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67ebf1085126812def359ef6'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'longi

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [91]:
# Change the data type from String to Decimal for longitude and latitude
pipeline = [
    # First ensure we have top-level latitude/longitude fields
    {"$set": {
        "latitude": {
            "$ifNull": [
                "$latitude",
                "$geocode.latitude",
                None
            ]
        },
        "longitude": {
            "$ifNull": [
                "$longitude",
                "$geocode.longitude",
                None
            ]
        }
    }},
    # Then convert any string coordinates to decimal
    {"$set": {
        "latitude": {
            "$cond": [
                {"$eq": [{"$type": "$latitude"}, "string"]},
                {"$convert": {
                    "input": "$latitude",
                    "to": "decimal",
                    "onError": None
                }},
                "$latitude"
            ]
        },
        "longitude": {
            "$cond": [
                {"$eq": [{"$type": "$longitude"}, "string"]},
                {"$convert": {
                    "input": "$longitude",
                    "to": "decimal",
                    "onError": None
                }},
                "$longitude"
            ]
        }
    }}
]

result = establishments.update_many({}, pipeline)

print(f"Matched {result.matched_count} documents")
print(f"Modified {result.modified_count} documents")


Matched 77584 documents
Modified 0 documents


Use `update_many` to convert `RatingValue` to integer numbers.

In [94]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [96]:
# Change the data type from String to Integer for RatingValue
pipeline = [
    {
        "$set": {
            "RatingValue": {
                "$cond": {
                    "if": { "$regexMatch": { "input": "$RatingValue", "regex": "^[0-9]+$" } },
                    "then": { "$toInt": "$RatingValue" },
                    "else": "$RatingValue"
                }
            }
        }
    }
]

# Execute the update
result = establishments.update_many(
    { "RatingValue": { "$type": "string" } },
    pipeline
)

print(f"Matched documents: {result.matched_count}")
print(f"Modified documents: {result.modified_count}")


Matched documents: 2
Modified documents: 0


In [98]:
# Check that the coordinates and rating value are now numbers
sample = establishments.find_one()
print({
    "latitude": type(sample.get("latitude")),
    "longitude": type(sample.get("longitude")),
    "RatingValue": type(sample.get("RatingValue"))
})


{'latitude': <class 'bson.decimal128.Decimal128'>, 'longitude': <class 'bson.decimal128.Decimal128'>, 'RatingValue': <class 'int'>}
